# Урок 1. Метрики качества классификации ч.2.

Рассмотрев в предыдущем уроке матрицу ошибок, мы выяснили, что каждый ее элемент означает конкретную ошибку для конкретного класса. Но что, если мы хотим оценить модель в общем, не привязываясь ни к одному из классов? Рассмотрим несколько метрик, которые позволяют это сделать.

### F-мера

Нас не всегда интересуют точность и полнота как отдельные меры качества работы алгоритма - иногда более важно оценить, как работает алгоритм в целом, поэтому было бы здорово получить один критерий качества, который бы представлял и точность, и полноту.

Можно взять среднее арифметическое $arithmetic\ mean = \frac{precision + recall}{2}$, но если алгоритм будет относить все объекты к положительному классу, то $precision \ll 1$ и $recall = 1$, а их среднее арифметическое будет больше 1/2, что явно лучше истинного положения.

Соответственно, мы нуждаемся в более сбалансированной мере, чем среднее арифметическое: нам нужна метрика, которая будет близка к 0, если хотя бы один из аргументов (точность или полнота) близок к 0. 

Одной из таких мер является *F-мера* - гармоническое среднее точности и полноты. Оно является своего рода средним, когда точность и полнота равны, но обладает важным свойством - среднее гармоническое близко к нулю, если хотя бы один из аргументов близок к нулю. 

$$
F_{\beta} = (1 + \beta^2) \cdot \frac{precision \cdot recall}{\beta^2 \cdot precision + recall}
$$

где $\beta$ определяет вес точности в метрике ($0 < \beta < 1$). При $\beta = 1$ это сбалансированная F-мера (ее также называют F1-мерой) - здесь точность и полнота имеют одинаковый вес.

Таким образом, если одно из чисел мало в сравнении с другим, F-мера становится ближе к меньшему значению, давая модели подходящую оценку.

![](https://248006.selcdn.ru/public/DS_Block2_M6_final/f_score.png)

### AUC-ROC

Все предыдущие метрики предполагали, что алгоритм однозначно определяет, к какому классу относится объект. Но некоторые алгоритмы дают на выходе вещественный ответ в пределах [0,1] (назовем его вероятностью того, что объект принадлежит данному классу), и затем он конвертируется в бинарную метку, основываясь на заранее заданном пороге, при котором 0 становится 1. Допустим, алгоритм определил, что электронное письмо с вероятностью *p* является спамом. Тогда при пороге в 0.7, если $p < 0.7$, письмо относится к классу *не спам*, а при $p \geq 0.7$ - к классу *спам*.

Одним из способов оценить модель в целом, не привязываясь к конкретному порогу, является *AUC-ROC* - площадь (Area Under Curve) под кривой ошибок (Receiver Operating Characteristic curve). Эта кривая представляет собой линию от (0,0) до (1,1) в координатах TPR (True Positive Rate) и FPR (False Positive Rate), которые определяются из соотношений:

$$
TPR = \frac{TP}{TP + FN}
$$

$$
FPR = \frac{FP}{FP + TN}
$$

Координата TPR - это полнота, а FPR показывает, какую долю из объектов класса 0 (negative) алгоритм предсказал неверно. В идеальном случае, когда классификатор не делает ошибок (FPR = 0, TPR = 1), мы получим площадь под кривой, равную единице. То есть чем больше площадь под кривой, тем лучше модель предсказывает класс объектов, при этом в идеале кривая должна стремиться к точке (0,1), т.к. мы хотим максимизировать TPR, минимизируя FPR. Диагональная прямая (*x = y*) соответствует полной неразличимости двух классов, то есть при AUC = 0.5 классификатор относит объекты класса 1 к классу 0, а объекты класса 0 - к классу 1.

![](https://248006.selcdn.ru/public/DS_Block2_M6_final/AUC-ROC.png)

Каждая точка пространства соответствует возможному порогу. Об этом подробнее можно прочитать [здесь](https://dyakonov.org/2017/07/28/auc-roc-%D0%BF%D0%BB%D0%BE%D1%89%D0%B0%D0%B4%D1%8C-%D0%BF%D0%BE%D0%B4-%D0%BA%D1%80%D0%B8%D0%B2%D0%BE%D0%B9-%D0%BE%D1%88%D0%B8%D0%B1%D0%BE%D0%BA/).

### Логистическая функция потерь (logloss)

Говоря о метриках, которые используются для алгоритмов, возвращающих вещественный ответ, также стоит упомянуть логистическую функцию потерь. Интуитивно можно представить минимизацию *logloss* как задачу максимизации *accuracy* путем штрафа за неверные предсказания: чем менее точно модель предсказывает верный класс объекта, тем больше становится логистическая функция потерь.

$$
logloss = -\frac{1}{l} \cdot \sum_{i=1}^{l} (y_i \cdot log(\hat{y}_i) + (1 - y_i) \cdot log(1 - \hat{y}_i))
$$

где $\hat{y}$ - ответ алгоритма на i-м объекте, y - фактическая метка класса, *l* - размер выборки.

Подробнее про эту функцию потерь можно прочитать в [этой публикации](https://dyakonov.org/2018/03/12/%D0%BB%D0%BE%D0%B3%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F-%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F-%D0%BE%D1%88%D0%B8%D0%B1%D0%BA%D0%B8/).

*Logloss* крайне сильно штрафует за уверенность классификатора в неверном ответе. Уверенность классификатора определяется тем, насколько близко к краям отрезка [0,1] он определяет вероятность отнесения объекта к тому или иному классу. Допустим, если вероятность равна 0.5, то это неуверенная классификация, а если 0.1 или 0.9 -  уверенная.

![](https://248006.selcdn.ru/public/DS_Block2_M6_final/logloss.png)

Видно, что чем ближе к нулю ответ алгоритма при верном классе *1*, тем выше значение ошибки.

Все указанные метрики реализованы в библиотеке [sklearn](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics).

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import log_loss